# Visualization of CDIAC-Data on global CO2 Emissions

This notebook is intended to visualize the data on global CO2 emissions from the "[carbon dioxide information and analysis center](https://cdiac.ess-dive.lbl.gov/)" (CDIAC) over the last centuries. It is inspired mainly as a means to scrutinze the claims given in the positional whitepaper [on animal agriculture and climate change](https://climatehealers.org/the-science/animal-agriculture-position-paper/) by stanford alumni Dr. Sailesh Rao. He therein claims, that the share of emissions caused by animal agriculture amounts to 87% of all anthropogenic green house gas (GHG) as a lower bound.  While this is a sheer unbelievably high amount on the one side, he makes some excellent points in his paper. Honoring, the royal societies motto "Nullius in verba" (latin for 'Take nobodys word for it') I wanted to check the presented data myself.

This python notebook is intended to check in particulary one of his points and scrutinize the data vizualized in figures 2.3 and 2.4 in the white paper. The text in this notebook can be followed without reading Dr Rao's paper. However, I would strongly reccomend reading it. 

The following python code is used to call the data from the cited sources and do the necessary processing to vizualize it.  Whereby presenting the code in this jupyter notebook also serves as means of transparency and tracability.

## Preparation and loading of data from web sources
First we import nessecary modules for loading, processing and displaying the data. [Pandas](https://pandas.pydata.org/) comes in very handy is a "fast, powerful, flexible and easy to use open source data analysis and manipulation tool". It has almost everything built in that we need. [plotly](https://plotly.com/python/) is a graphing library intened for web-grade data visualisation, with  a mouse-over readout function.

In [1]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import urllib
import numpy as np
import ssl

pd.options.plotting.backend = "plotly"
ssl._create_default_https_context = ssl._create_unverified_context # needed for ssl connections

We call the data directly from the online sources given in the following URLs from [CDIAC](https://cdiac.ess-dive.lbl.gov/). CDIAC is a governmental data archive on the issue of climate change by the US government. The data is currently being transitioned to U.S. Department of Energy’s (DOE) Environmental System Scienc, but for the time being we can collect the data from CDIAC.

The URL below is taken from the hyperlink to the "Latest Published Global Estimates" on [this page](https://cdiac.ess-dive.lbl.gov/trends/emis/meth_reg.html). An inspection of the file header (the first 37 lines of the file from the URL printed below) is useful to examine the data structure and units of the data acquisition. It also gives the authors credit.

In [2]:
url_fossils = "https://cdiac.ess-dive.lbl.gov/ftp/ndp030/global.1751_2014.ems"

In [3]:
# print header i.e. first couple lines of txt file with some data
fossil_rawfile = urllib.request.urlopen(url_fossils)
txt_lines = fossil_rawfile.readlines()
for line in zip(range(37),txt_lines):
    print(line[1][:-1].decode('utf-8'))

***********************************************************
*** Global CO2 Emissions from Fossil-Fuel Burning,      ***
*** Cement Manufacture, and Gas Flaring: 1751-2014      ***
***                                                     ***
*** March 3, 2017                                       ***
***                                                     ***
*** Source:  Tom Boden                                  ***
***          Bob Andres                                 ***
***          Carbon Dioxide Information Analysis Center ***
***          Oak Ridge National Laboratory              ***
***          Oak Ridge, Tennessee 37831-6290            ***
***          USA                                        ***
***                                                     ***
***          Gregg Marland                              ***
***          Research Institute for Environment, Energy ***
***            and Economics                            ***
***          Appalachian State Universit

We shall also inspect the header of the land use data, which is taken from [this website](https://cdiac.ess-dive.lbl.gov/trends/landuse/houghton/houghton.html). More information on the methods of assesment of the data can be found in the original [article](https://onlinelibrary.wiley.com/doi/full/10.1034/j.1600-0889.2003.01450.x) by Houghten et. al in the Wiley Online Library. 

In [4]:
url_landuse = "https://cdiac.ess-dive.lbl.gov/trends/landuse/houghton/Global_land-use_flux-1850_2005.xls"

In [5]:
# print header i.e. first couple lines of txt file with some data
fossil_rawfile = urllib.request.urlopen("https://cdiac.ess-dive.lbl.gov/trends/landuse/houghton/1850-2005.txt")
txt_lines = fossil_rawfile.readlines()
for line in zip(range(25),txt_lines):
    print(line[1][:-1].decode('utf-8'))

Annual Net Flux of Carbon to the Atmosphere from Land-Use Change:  1850-2005
R.A. Houghton

Units = Tg C (1 teragram = 10^12 g)

Positive values represent a release of carbon to the atmosphere, while
negative values represent a net uptake of carbon by the terrestrial 
biosphere.

Please note:  due to rounding, the sum of the regions included in this textfile may not equal the global
 totals listed in the TRENDS document.  Please refer to the Excel spreadsheet for more precise estimates.

 Year   Global    USA    Canada   South+   Europe   N.Africa Tropical  Former  China   South+     Pacific
                                 Central            +Middle   Africa   USSR           Southeast  Developed
                                 America              East                               Asia     Region
 1850   500.6    164.1     5.5     23.5     55.0      4.0     -1.3     58.6    101.8     87.3      2.0
 1851   492.7    165.7     5.4     23.2     55.0      4.0     -1.1     58.6     93.1  

Teragrams ($10^{12}$ grams) and million metric tons are the same as one million grams amounts to a ton.  Also a positive carbon flux to the atmosphere and carbon emissions are the same, as both describe the yearly addition of carbon atoms to the atmosphere. Note that we can also simply multiply the  data by a factor of 3.667 to obtain the units of CO2 as stated in the header from the first file. (A CO2 molecule has 3.667 times the mass of a carbon atom.)

## Import
So let's import the  data into two actual pandas dataframes ...

In [6]:
# import data in million metric tonnes carbon per Year
co2_fossil_emissions = pd.read_csv(url_fossils, skiprows=33, sep='\s+', index_col=0)

In [ ]:
# data: 10^12 g is million metric tonnes carbon per year
co2flux_from_landuse = pd.read_excel(url_landuse,sheet_name='net fluxes',index_col='Year')

## Inspection 
It is helpful to inspect the imported dataframes, which can be done by calling it with the `.head()` or `.tail()` methods, to show either he beginning or the end of the frame. This is particularly handy for large data sets. 

In [ ]:
co2_fossil_emissions.tail()

When skipping the first 33 lines during the pandas import, we did cut of part of the columns names. We can fix that with a dictonary to replace the column names above with the key values from a dictionary like so:

In [ ]:
# better readabillity for labels in plots, also fix broken labels from truncated txt-file
coldict={'Production':'Cement', 'Solids':'Coal', 'Liquids': 'Oil', 'Total': 'Subtotal'}
co2_fossil_emissions.rename(columns=coldict, inplace=True)

In [ ]:
co2_fossil_emissions.tail()

Here I took the liberty to also rename 'Solids' to 'Coal' and 'Liquids' to 'Oil', which is essentially what they are. 

Let's also inspect the dataframe on CO2 flux from land use:

In [ ]:
co2flux_from_landuse.tail()

It is noteworthy that at the beginning of our century the US, Europe and China already sequester more carbon by land use than they emit by land use. This means that woods and nature are seemingly growing more than they are cut in forestry. This is of course quite positive, but the 12 to 32 million tons of sequestration per year can hardly compensate the output of billion tons of fossil carbon. Also the negative values have a political reason with respect to globalisation, that I will discuss at the end of the document.

## Processing 
We do the conversion of the data in the datafames from carbon to CO2 as stated above. We also devide by 1000 to get the datafame entries in billion tons.  We'll get into these dimensions quickly anyway and it makes tick labels easier to read.

In [ ]:
co2flux_from_landuse = co2flux_from_landuse.applymap(lambda x: x * 3.667 * 1e-3)
co2_fossil_emissions = co2_fossil_emissions.applymap(lambda x: x * 3.667 * 1e-3)

We are not intersted in a distiction between gas and flaring gas, so we just add them together. An get rid of the per capita information as well.

In [ ]:
# sum gas and flaring gas emissions to total gas
co2_fossil_emissions['Gas'] = co2_fossil_emissions['Gas'] + co2_fossil_emissions['Flaring']
# drop unneeded columns
co2_fossil_emissions.drop(columns=['Capita', 'Flaring'],inplace=True)

For ease of use, we combine the two datafames into one. `co2_emission` is just holding the gobal emissions, where each column shows the emissions of CO2 in billion metric tons, with regard to the various carbon sources per year. Also for the total gobal emissions, we want to include the emissions by land use, so we have to add the gobal CO2 flux from land use to the fossil emissions in order to gain the new total.

In [ ]:
# joint dataframe for fossil and landuse data
co2_emissions = co2_fossil_emissions.copy()
co2_emissions['Landuse'] = co2flux_from_landuse['Global'] # just the gobal data is relevant for comparison
co2_emissions['Total'] = co2_fossil_emissions['Subtotal'] + co2flux_from_landuse['Global']

Let's check the years 2001 to 2011:

In [ ]:
co2_emissions = co2_emissions[['Gas', 'Oil', 'Coal', 'Cement','Subtotal', 'Landuse', 'Total']]

In [ ]:
co2_emissions.loc[2001:2011]

40 Billion tons of total emissions including land use in 2005 is in line with the data we have from the [IPCC](https://www.ipcc.ch/). If we substract land use again we get about 34 gigatons (= billion tons), which is what is reported by [other sources](https://de.statista.com/statistik/daten/studie/37187/umfrage/der-weltweite-co2-ausstoss-seit-1751/) as well. But those are not the datasets we are looking at. By the way in 2020 emissions amounted to 36 gigatons from fossils according to the IPCC, so we are not doing a very good job at cutting emissions. 

Looking at the data above again, note in 2005 we emitted worldwide double the amount of CO2 from coal or gas than we emitted from land use or gas. This is particularly notworthy if we look at the cumulated shares later.

For some years  we don't have data available (`NaN` = not a number) but that's ok. Also for the  sake of the  argument that we are going to make it is not really important if the estimates represtend by this data set are off by a few billions tons. 

Instead of asking the question of how much  anthropogenic CO2 we put into the atmosphere yearly we can also ask, how much of the CO2 ends up staying there due to the various sources. In other words: In order to reduce emissions, which source would be the most effective to focus the cut on. For this we have to know two things: First, roughly 65% of all of the airborne CO2 gets sequestered by surface-plants, algae (e.g. diatoms), coral (or precisely the symbiotic algea living within the coral) and the micro-biome living in the soil each year. That means only 45% of the CO2 let into the atmosphere stays there. Dr. Rao calls this the airborn fraction in his article. Secondly, non-sequestered stays in the atmosphere basically forever. Energetically CO2 is a very stable molecule, that needs the suns energy and specialized enzymes to be chemically altered (e.g. in photosynthesis). Also it has a realatively high mass, so that dissipation into open space in the upper atmosphere is negligible. 

In consequence to gain an estimate for the antropogenic CO2 in the atmosphere, we just have to sum the emissions for each of the sources and multiply by 0.45 (distributive law).

In [ ]:
co2_cumulated_emissions = co2_emissions.cumsum()
co2_atmospheric_remainder = co2_cumulated_emissions.applymap(lambda x: x * 0.45)

In [ ]:
co2_atmospheric_remainder.loc[2001:2011]

Summation with a "NaN" as one of the summands amounts to another NaN as a result.

## Visualisation
Let's plot using the plotly syntax directly from the pandas dataframe object to get a better picture. NaN-Values are not plotted.

In [ ]:
fig1 = co2_emissions.plot(labels=dict(index="Year", value="CO2-Emission [Billion Tons per Year]",
                                      variable="Source"),
                          template='none')
fig1.update_xaxes(range=[1750, 2020]);
fig1.update_layout(
    font=dict(size=14),
    legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.05,
    title='Source:'
    ),
    template='plotly_dark');
fig1.show()

In [ ]:
fig1.write_image(co2_emissions_per_sources.png)

This doesn't quite look like figure 2.3 in Dr. Raos whitepaper. Of couse the abscissa is starting in 1750, but other than that the lines are crossing each other here.

To achieve more resemblance we need a slightly different representation where the emissions are represented in a so-called area plot, which can be realized like follows:

In [ ]:
new_cols = ['Landuse', 'Coal', 'Oil', 'Gas', 'Cement'] 
co2_emissions_sorted = co2_emissions[new_cols] #new DF with colums in order of new_cols

# sum along colums axis to get total emissions per year
co2_emissions_sorted_sum = co2_emissions_sorted.cumsum(axis=1) 
co2_emissions_sorted_sum.dropna(inplace=True)

In [ ]:
fig2 = go.Figure()
last_col = None
cols = co2_emissions_sorted_sum.columns
for col in cols:
    if last_col is None:
        cursor_value = co2_emissions_sorted_sum[col]
    else:
        cursor_value = co2_emissions_sorted_sum[col] - co2_emissions_sorted_sum[last_col]
    fig2.add_trace(go.Scatter(x = co2_emissions_sorted_sum.index.values, y = co2_emissions_sorted_sum[col],
                              fill='tonexty', mode= 'none', name=col, text=cursor_value,
                              hovertemplate = '%{x}: %{text:.2f} gigatons/year'))
    last_col = col
    
fig2.update_xaxes(title='Year');
fig2.update_yaxes(title='CO2 emission [billion tons per year] ');
fig2.update_layout(template='none')
fig2.update_layout(
    font=dict(size=14),
    legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.05,
    title='Source:'
    ),
    template='plotly_dark'
)
fig2.show()

In [ ]:
fig1.write_html("co2_emissions_area_plot.png")

A comparison of aboves image to the figure 2.3 in Dr. Rao's whitepaper shows that it is indeed in line with the original data from CDIAC. 
Personally I find this repsenstation in an area plot quite confusing. If one is not familiar with this kind of plot, one might read them wrong. E.g. looking at the ordinate for the year 2000 for coal, one might think that the emission value for coal was roughly 14 billio tons, when it actually was 8,5 billion tons. The code above is written so, that the actual emission value per year can be read out with the mouse-cursor during a mouse-hover event on the specific coordinate using plotly. Fig2 may look nicer graphically, yet I still prefer the representation in fig1 for clarity.

BTW: The image in fig2 could more simply be achieved by calling `fig4 = co2_emissions_sorted[new_cols].plot.area(...)`. But through the code above we can understand better that the emission value for each source is actually the difference between two color boundries. We can alreay see from the colored areas in fig2, that the amount of CO2 from land use takes more area than that from burning gas for instance.

As discussed above the data in the data frame `co2_atmospheric_remainder` represents the amount of CO2 which remains in the atmosphere each year.

In [ ]:
fig3 = co2_atmospheric_remainder.plot(labels=dict(index="Year",
                                                value="anthropogenic CO2 in atmosphere [billion tons]",
                                                variable="Source"),
                                      template='none')
fig3.update_xaxes(range=[1800, 2010]);
fig3.update_layout(
    font=dict(size=14),
    legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.05,
    title='Source:'
    ),
    template='plotly_dark'
    )
fig3.show()

In [ ]:
fig2.write_html("anthropogenic_co2_in_asmosphere.png")

The plot from fig3 shows that the mount of CO2 in the atmosphere from land use exeeded that of all other sources up until the year 2005, and is roughly the same for the following years. This is also in line with Dr. Raos argumentation on figure 2.4 in his whitepaper. The absolute values on the abscissa differ only because we decided to use the factor of 0.45 on the data frame, while Dr. Rao shows the cumulated emissions. 
His claims concerning the share of land use in global CO2-emissions are therefore confirmed.

## Interpretation
According to the underlying data historical emissions due to coal and land-use amount to 250 billion tons and roughly *on third* of all anthropogenic GHGs in the atmosphere today! With only 160 billion tons of CO2 from oil, land-use even has a far graver impact than all burning of oil, which includes the whole transportation sector, including airplanes.

Keep in mind, that within CDIAC's data set, the emissions from land use are not available for the years before 1850. Hence, there sum amounts to zero, although humankind has been pusruing agriculture and land use for far longer periods than it has been burning coal. Indeed the estimate is that the agricultural revolution began roughly 8000 years ago, while the industrialisation began little over 200 years ago. We can actually see it in the chart in fig2 with the onset of coal emissions in 1850.  

If we were to extrapolate or raise the data on land use emissions before 1850 it could be easily argued, that land use would remain the biggest chunk of GHGs in the atmosphere even today. In fact, there is [evidence](https://www.youtube.com/watch?v=1TOTsmqgmL8) that global temperatures began to rise even earlier than the industrial revolution.

This makes a strong case for those who say that a system change in agriculture is at least equally as important as a system change in energy industry and the transportation sector, because agriculture is the main contributor to land use.

On the other hand we saw that yearly emissions due to coal or oil exceed the emissions from land use by a factor of two. Only by cumulating past emissions became land use comparable to coal. Yet we can't change what is in the past. We can only change current emissions. Wouldn'it make more sense then, to tackle these big contributors first? 

### Sequestration potential of natural habitats and ecosystems 

No! To address the climate crisis all net carbon flow to the atmosphere from athropogenic sources must be stopped. Land use were still adding a whopping 5 billion tons of CO2, each year, if we stopped everything else but. Yet, as Dr. Rao points out in the article, something else is more important. Land use is the only carbon source than can actually serve as a sink instead. A good estimate of its potential is currently the subject of scientific debate. Much of the debate consider eco systems like forests of soil organic carbon in agriculture individually. Each of them alone has the potential to sequester all of the emitted fossil carbon, with more or less certainty.

E.g. in [this article](https://www.sciencedirect.com/science/article/pii/S0016706117300095) it is discussed that carbon could be sequstered sufficiently on cropland. Sequestration rates of 0.2 to 0.5 tonnes of carbon per year and total land ara in hectar are suppsedly possible, if we use less intrusive methods of agriculture in crop lands (e.g. reduced tilling). Calculating from these numbers we could cancel at about 37 billion tons of CO2 emissions per year. That's already more than our current fossil emissions, just by changing the way we treat our cropland. This is not even considering biodiversity effects. How much more carbon could be sequestered if we left nature restore its natural habitats?

At least 80% of all global farm land is used to raise animal feed, while animals contribute only to [13% of calories consumed by humans](https://ourworldindata.org/agricultural-land-by-global-diets). This is a clear waste of area, which could otherwise serve as natural habitat and help stablize the natrual carbon cycle. In fact, agricultural monocultures are one of the main drivers for the loss of biodiversity and towards the 6th major mass extinction. 

Concerning forrests: We know [from literature](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1466-8238.2010.00587.x) that currently 35 millon km$^2$ and 25 million km$^2$ are used for cropland and pasture respectively, most of which have been cleared from forrests. And from the [FAO](http://www.fao.org/3/y0900e/y0900e06.htm) that forrest have a carbon sequestration potential depending on their region, ranging from 0.8 to 2.4 tons of carbon per year and hectar in boreal forrets and 3.2 to 10 tons for tropical forrest. Restoring 70% of that area, leaving 30% to grow food for 14 billion people the CO2 sequestration range would somwhere near 38 billion tons of CO2. Again, more than we currently emit.

In [ ]:
pasture_sqkm, cropland_sqkm = 30e6, 20e6
restored_forest_area_ha = (pasture_sqkm * 100 + cropland_sqkm * 100)* 0.7
["{:.2f} ".format(carbon_seq_rate * restored_forest_area_ha / 1e9 * 3.667) for carbon_seq_rate in (0.8,3,10)]

But forests and soil are not the only means so absorb carbon. As I said, the main stabilizing element for climate is natures diversity. Proving that is  well beyond that scope of a text like this. Sir Davig Attenborough makes some excellent points in his documentary [My life on this planet](https://www.netflix.com/watch/80216393). It may serve as a straing point for personal investiagtions for the reader. 
There is also a particular example from another series, that I like. I goes so show that symbiotic relationsships between ecosystems can extend by thousands of miles stretching between realms: Have you heard of the stream in the [clouds that's feeding the diatoms in the sea](https://youtu.be/vzRkHkssdME)? In short: Evaporation of water from the amazon rain forest, builds up to form clouds just at the top of the canopy, which drift west and then rain down at the andes. There the water washes out sediment, runs downstream though the amazon basin, taking up more nutrients bringing them into the sea, where they feed diatom algae. Those in turn photosynthesize carbon and produce oxygen and water. That is 50% of all oxygen in our atmosphre stems from diatoms all over the world not forests. In another instance salmon when they swim umstream into the forests of the northern hemisphere, in their death fertilize the fores by bringing nutriens from the sea. The northern terretories in Canada are dying because by fishing salmon on a large sacle we ware preventing that nutrients to reach these terretories. Which in turn comporises the northern woods in their capability as much carbon as they would, when they were healthy. Here's a [german speaking documentation](https://www.zdf.de/dokumentation/dokumentation-sonstige/hannes-jaenicke-im-einsatz-fuer-den-lachs-100.html) on that subject.

Diatom's carbon sequestration capablity is estimated to be [100 to 250 tons CO2 per hecar per year](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7464044/). That amount's to 5 to 10 Gigatons of carbon squestered only from the diatoms of the coast of brasil. Yet we are breaking that cycle by clearing the amazon and hindering the cloud stream from feeding the diatoms and expanding their full sequestration potential. 

This is just an anectodal example of the complex relations of ecosystems that happen everywhere in world. We haven't even considered coral, phytoplancton or soil organic crabon in non-agricultural areas. Each of them having a huge carbon binding potential on their own, if we only left them undisturbed. And almost all of them are disturbed by agriculture, our food system and the "klilling machine".  Estimates see yearly sequestion rates above 200 Gigatons for a world in which we stop "the klilling" machine and the exploitation of animals.

Let's see what that meens for atmosphereic levels extending the for years until 2050 into the future:

In [ ]:
co2_projected = co2_emissions.reindex(range(1751,2051))

But renaturated land and oceans and let them play out their full carbon binding potential. Let's check for various  rates, say 18, 36 or 72 billion tons per year. All of them well within the ranges discussed above

In [ ]:
sequestrion_rate_examples = [-18,-36,-72]

In [ ]:
multi_index = pd.MultiIndex.from_product([sequestrion_rate_examples,range(1751,2051)],names=['Seq_Rate_Gt_per_year', 'Year'])
cols = ['Gas', 'Oil', 'Coal', 'Cement','Landuse']
co2_projected = pd.DataFrame(index=multi_index, columns=cols)
co2_projected.sort_index(level=['Seq_Rate_Gt_per_year','Year'], ascending=[1, 1], inplace=True)

Assuming we cept fossil emissions roughly at the current level... 

In [ ]:
for seq_rate in sequestrion_rate_examples:
    for col in cols:
        co2_projected.loc[(seq_rate,slice(1751,2014)), col] = co2_emissions[col].values
        co2_projected.loc[(seq_rate,slice(2015,2050)), col] = co2_emissions.loc[2014, col]
    co2_projected.loc[(seq_rate,slice(2006,2050)),'Landuse'] = seq_rate

and calculate the new negative totals

In [ ]:
co2_projected['Total'] = co2_projected.sum(axis=1)

In [ ]:
co2_projected_sum = co2_projected.groupby(['Seq_Rate_Gt_per_year']).apply(lambda x: x.cumsum() * 0.45)

In [ ]:
fig5 = go.Figure()
for seq_rate in sequestrion_rate_examples:
    fig5.add_trace(go.Scatter(x = co2_projected_sum.index.levels[1], y = co2_projected_sum.loc[(seq_rate,slice(None)),'Total'],
                              mode= 'lines', name=seq_rate, text=cursor_value))
    
fig5.update_xaxes(title='Year',range=(1800,2050));
fig5.update_yaxes(title='anthropogenic CO2 in atmosphere [billion tons]');
fig5.update_layout(template='none')
fig5.update_layout(
    font=dict(size=14),
    legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.05,
    title='sequestration rate [Gt/a]'
    ),
    template='plotly_dark')
fig5.show()

In [ ]:
fig5.write_image('anthropogenic_CO2_sequestrion_potential_sczenarios.png')

As we can see in the green and yellow curves, we have the potential not only cancel emissions to a net zero with constant levels. We are also able to reduce CO2 levels to values we measured from before the onset of industrialization. And all of that is possible by just reviving natural ecosystems, even while continuing fossil emissions at current levels. This is not to say that we could or should get away with continuing fossil emissions. What it should show is, that current policies can only bring us so far. They can only bring us to a net-zero. Yet, things are already bleak for a lot of people in the global south even today. There is already suffering. Reducing emissions to net zero only stops things from getting even worse. That's why personally I get quite upset when politicians speak of a CO2 budget. There is no budget! A budget is somthing that you can spend from at your own  expenses. In reality we are in a loan-type of situation with debts way over our head.
But by letting nature and biodiversity thrive, we can actually  follow the green curve, improve conditions for people around the globe and pay our debt back. 

Think about it: a damage that has slowly been done over a 200 to 2000 year time frame could be mitigated within the next 20 to 30 years! Most of us could live to see that happen. 
It's almost like a the body of a smoker. Long time damage to a smoker's lung can be [fully reverted]() by quitting damaging themselves day after day. The body heals itself, if we let it. In that sense our planet behaves similarly. Provided that it's not already to late and the "cancer" has not already spread. The striking difference is, that  with GHGs and ecosystems we have an actual means to take the harmful substance out of the system, while the body just accomplishes that, if we stop putting it there. This fact cannot be stressed enough!

Of course the  sequestration rates used in the example above are rather wild guesses, and were deliberatly chosen to showcase the discussed scenarios.  But as pointed out before their order of magitude is well grounded in evidence and therefore within our grasp.
In fact our best current estimates show, that carbon is sequestered by at least [2400 gigatons each year by soil alone](https://doi.org/10.1016/j.geoderma.2017.01.002). That means that an increas of  nature's sequestration rate by a demanded 72 gigatons per year, relates to an increase of not more than 3%. Yet it seems to be exactly that kind of percentage by which our species is hindering the biosphere to absorbe what is excessly emitted. Again, if we let the green biosphere grow and be diverse instead of cutting it down for animal feed year after year, we would quite possibly be on constant rather than rising CO2-levels.
Similar ideas have already been proposed by french minister of agriculture Stéphane Le Foll at the 2015 climate summit. It was called the 4-per-mille-initiative as the nessecary increase at that time was only 0.4%. But the initiative's goal was only cancellation (yellow curve) rather than reversal (green curve).

All of this goes to show, that the best means to tackle the climate crisis as a species is how we treat animals and use the land and the oceans.
If only we had the will to do what is right? Isn't it striking that the ethically right thing to do just so happens to be the same thing that can save us of from living in droughts and food shortages and suffering from wars of distribution and mass migrations?

I don't believe that needed change in agriculture will be induced by policy makers. They don't even see the problem. In the land use data we saw that Europe, the US and China showed negative emissions, while total land use emissions where positive. So these governments will consider their slate clean.
Of course, the assumption that these three did not contribute to climate change trough land use is still wrong. Due to globalisation and market effects cheep animal feed (i.e. soy or oil plants) is bought mainly from south america, by exactly those three.
But the amount of imported feed and therefor the used area is only dictated by the demand for meat and dairy products - by us! 

The disussion so far only looked at carbon emissions, both in fossil emissions and opportunity cost from land-use. When cosidering what climate scientists call the radative forcing of all GHGs including methane, the evidence, that agriculture is the main driver behind and at the same time best means to avert the climate crisis, becomes even more compelling. Dr. Rao explains this in his whitepaper and clearly shows with his linear sensitvity analysis, that there is scientific bias even in the international panel on climate change (IPCC). I will not go into methane sequestration mechanisms or the details on radiative forcing. However, methane is primarly caused by agriculture, which is again supported by consumer demand for animal products.

Hence, consumers hold the true power to turn around climate change by simply rethinking or reinventing their taste. The only problem is, that everybody need to do their part. Personally, I'd rather see a vegan world than a non-vegan world. Mostly becaus of the evolution of my morality. But from a climate system's point-of-view there is no black and white. If this text goes to show anything, than that reducing meat and animal consumption is far more effective than refraining from long distance flights or optimizing or reducing our heating and electricity.

